In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd


In [2]:
datasig=pd.read_csv("sig_many_values.csv")
databkg=pd.read_csv("bkg_many_values.csv")

In [3]:
datasig.shape

(67263, 57)

In [4]:
databkg.shape

(84499, 57)

In [5]:
databkg.iloc[2:100,4];

In [6]:
datasig["label"]=1
databkg["label"]=2

In [7]:
databkg.shape

(84499, 58)

In [8]:
data2=datasig.append(databkg,ignore_index=True)

In [9]:
data2.shape

(151762, 58)

In [10]:
X,Y=data2.iloc[:,:-2],data2.iloc[:,-1]
weights=data2.iloc[:,-2]

In [11]:
import xgboost as xgb

In [12]:
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=7)
X_train, X_test, y_train, y_test,weight_train,weight_test= train_test_split(X, Y,weights,test_size=0.3, random_state=7)

In [13]:
xg_class= xgb.XGBClassifier(objective ='reg:logistic', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 8, alpha = 10, n_estimators = 10)
# xg_class= xgb.XGBClassifier()


In [14]:
weight_train=weight_train*10**4
weight_test=weight_test*10**4
import numpy as np
weight_train=np.asarray(weight_train)
weight_test=np.asarray(weight_test)
dtest = xgb.DMatrix(data=X_test,weight=weight_test)

In [15]:
#xg_class= XGBClassifier()

In [33]:
xg_class.fit(X_train, y_train,sample_weight=weight_train)
preds=xg_class.predict(X_test)

In [28]:
predictions = [round(value) for value in preds]
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 64.40%


In [29]:
preds

array([1, 2, 2, ..., 2, 1, 1])

In [30]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot as plt

In [34]:
probs = xg_class.predict_proba(X_test)
# keep probabilities for the positive outcome only
probs = probs[:, 1]
# calculate AUC
auc = roc_auc_score(y_test, probs,sample_weight=weight_test)
print('AUC: %.3f' % auc)
# calculate roc curve
fpr, tpr, thresholds = roc_curve(y_test, probs)
# plot the roc curve for the model
plt.figure(figsize=(8,8))
plt.plot(tpr, 1-fpr,linewidth=2)
# show the plot
plt.xlabel("sensitivity",fontsize=14)
plt.ylabel("specificity",fontsize=14)
plt.tick_params(axis="both",labelsize=14)
plt.show()

AUC: 0.687


ValueError: Data is not binary and pos_label is not specified